# Clasificador Naive Bayes de noticias argentinas

<a target="_blank" href="https://colab.research.google.com/github/pdomins/bayesian-learning/blob/master/ej2_bayes_news.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

El objetivo de este ejercicio es implementar un clasificador de texto utilizando el **clasificador ingenuo de Bayes** sobre el
conjunto de datos *”Noticias Argentinas”* para clasificar cada noticia según su tipo.

Librerías utilizadas:

In [1]:
import pandas as pd
import numpy as np
import nltk
import math

from nltk.corpus import stopwords

## Análisis del dataset

En principio contamos con 164690 tuplas:

In [2]:
df = pd.read_excel("Noticias_argentinas_clean.xlsx")
df

,fecha,titular,fuente,categoria
0,2018-12-13 15:49:06,Se van los Melli,NaN,Noticias destacadas
1,2018-12-26 21:21:41,Cantos racistas en el Calcio,NaN,Noticias destacadas
2,2018-12-26 21:21:41,Cantos racistas en el Calcio,NaN,Noticias destacadas
3,2019-01-13 16:35:30,Los que viajan a Uruguay son...,NaN,Noticias destacadas
4,2019-01-13 16:35:30,Los que viajan a Uruguay son...,NaN,Noticias destacadas
...,...,...,...,...
164685,2018-11-26 11:34:11,River Boca: el Gobierno nacional pide â€œinves...,NaN,NaN
164686,2018-11-26 11:34:11,River Boca: el Gobierno nacional pide â€œinves...,NaN,NaN
164687,2018-11-24 22:25:24,Se postergó San Lorenzo Huracán: el resto de l...,NaN,NaN
164688,2018-11-24 22:25:24,Se postergó San Lorenzo Huracán: el resto de l...,NaN,NaN


In [3]:
df['fuente'].unique().shape

(958,)

Al enumerar las categorías nos encontramos con el valor NaN, indicando que hay tuplas sin especificar su categoría:

In [4]:
df['categoria'].unique()

array(['Noticias destacadas', 'Ciencia y Tecnologia', nan, 'Deportes',
       'Entretenimiento', 'Destacadas', 'Actualidad', 'Crítica'],
      dtype=object)

Al contar los valores por cada una (sin vacíos):

In [5]:
print(df.groupby('categoria')['titular'].count())

categoria
Actualidad                   1
Ciencia y Tecnologia      2966
Crítica                      4
Deportes                  2969
Destacadas                2971
Entretenimiento           2961
Noticias destacadas     133864
Name: titular, dtype: int64


Además, al observar el dataset más arriba, podemos ver que existen tuplas que se encuentran repetidas. Teniendo esto en cuenta, contamos nuevamente las noticias:

In [6]:
print(df.groupby('categoria')['titular'].nunique())

categoria
Actualidad                  1
Ciencia y Tecnologia      710
Crítica                     1
Deportes                 1402
Destacadas               1731
Entretenimiento          1199
Noticias destacadas     39491
Name: titular, dtype: int64


## División del conjunto de textos

Utilizamos el método K-Fold para futura cross-validation:

In [7]:
def k_fold_split(df : pd.DataFrame, k : int) -> tuple[pd.DataFrame, pd.DataFrame]:
    if (k < 2) : raise ValueError("k must be >= 2. The value of k was: {}".format(k))
    df         = df.copy()
    df         = df.sample(frac=1)
    df_size    = df.shape[0]
    fold_times = math.ceil(df_size / k)
    train_df   = pd.DataFrame(columns=df.columns)
    test_df    = pd.DataFrame(columns=df.columns)
    for i in range(fold_times):
        curr_fold = df.iloc[i*k:(i+1)*k]
        train_df  = pd.concat([train_df, curr_fold.iloc[0:k-1]])
        test_df   = pd.concat([test_df,  curr_fold.iloc[k-1].to_frame().T]) \
                    if curr_fold.shape[0] >= k else test_df
    return train_df, test_df

Vamos a crear dos conjuntos, uno de entrenamiento y otro de testeo, por lo que nos queda:

In [8]:
k = 2

In [9]:
test_size = df.shape[0] / k
(test_size * (k-1), test_size)

(82345.0, 82345.0)

In [10]:
train_df, test_df = k_fold_split(df, k)

## ❌ BORRAR SECCIÓN
Achico train_df para que no me explote el notebook

In [11]:
original_train_df = train_df
train_df = pd.DataFrame(train_df[:2500])

## Preprocesamiento de los datos

Vamos a expresar los títulos como un array conformado por sus palabras relevantes lematizadas:

In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
%%capture
!python -m spacy download es_core_news_md

In [14]:
import spacy
nlp = spacy.load("es_core_news_md")

In [15]:
def preprocess_text(title_string : str):

  # removing unnecesary symbols and tokenization
  title = nlp(title_string.strip('.,+-#:;¿?¡!"\''))

  lemmas = []

  for tok in title:
    word = tok.lemma_.lower()
    if word not in stop_words:
      lemmas.append(word)

  return lemmas

In [16]:
train_df['titular'] = train_df['titular'].apply(preprocess_text)
train_df.head()

,fecha,titular,fuente,categoria
48491,2019-07-02 23:04:46,"[empezar, conocer él, parte, letra, chico, acu...",El Cronista Comercial,Noticias destacadas
28474,2019-05-17 23:42:00,"[efecto, contagio, ser, clave, jaguares]",Clarín.com,Noticias destacadas
77377,2019-04-12 08:36:56,"[arrancar, copa, superliga, :, formato, ,, cru...",Infobae.com,Noticias destacadas
130442,2019-01-19 10:28:58,"[pasar, día, después, hallazgo, cuerpo, albert...",Perfil.com,Noticias destacadas
53418,2019-04-17 12:18:00,"[famoso, mostrar, indignadísima, foto, "", súpe...",El Intransigente,Noticias destacadas


## Armado de vocabulario

Para empezar creamos un diccionario con todas las palabras utilizadas en los titulares para cada categoría, evitando repetidos en cada set:

In [17]:
categories = train_df['categoria'].unique()
categories

array(['Noticias destacadas', nan, 'Ciencia y Tecnologia', 'Deportes',
       'Entretenimiento', 'Destacadas'], dtype=object)

In [18]:
vocab = {}

for category in categories:

  # set of words used in the current category
  cat_vocab = set()

  # subset of training data of the current category
  cat_train_df = train_df[train_df['categoria'] == category]

  for title in cat_train_df['titular']:
    cat_vocab.update(title)

  vocab[category] = list(cat_vocab)

Por ejemplo, para deportes:

In [19]:
dep_idx = np.where(categories == 'Deportes')[0][0]
vocab[categories[dep_idx]][:10]

['atlético',
 'aldosivi',
 'colón-belgrano',
 'floresta',
 'guardar',
 'fassi',
 'evitar',
 'vivas',
 'jugador',
 'jugar']

## Cálculo de frecuencias

Iteramos los titulares dentro de cada categoría y calculamos la frecuencia de aparición de las palabras dentro de cada categoría:

In [20]:
vocab_freq = {}
titles_by_cat = {}

for cat_idx, category in enumerate(categories):

  # all the words used the in current category
  word_list = vocab[categories[cat_idx]]

  # dict with the frequencies of every word in the current category
  words_freq = { key: 0 for key in word_list }

  # training data subset of the current category
  cat_train_df = train_df[train_df['categoria'] == category]

  # amount of titles in the current category
  cat_title_count = len(cat_train_df['titular'])
  titles_by_cat[category] = cat_title_count

  for title in cat_train_df['titular']:
    for word in word_list:
      if word in title:
        words_freq[word] += (1 / cat_title_count)

  vocab_freq[category] = words_freq

## Cálculo de probabilidades

Primero calculamos la probabilidad de que un titular pertenezca a cierta categoría:

In [27]:
title_count = sum(titles_by_cat.values())
p_cat = { key: 0 for key in categories }

for category in categories:
  p_cat[category] = titles_by_cat[category] / title_count

p_cat

{'Noticias destacadas': 0.908390022675737,
 nan: 0.0,
 'Ciencia y Tecnologia': 0.026303854875283448,
 'Deportes': 0.019954648526077097,
 'Entretenimiento': 0.024489795918367346,
 'Destacadas': 0.020861678004535148}

In [23]:
sum(p_cat.values())

1.0

Con esta información y con las frecuencias guardadas en la sección anterior podemos calcular **P(A|categoria)**, es decir, la probabilidad de ocurrencia de un conjunto de palabras dada cierta categoría:

In [34]:
def p_a_cat(conj_a : list, category : str):
  prob = 1
  words_freq = vocab_freq[category]
  for word in conj_a:
    freq = words_freq.get(word)
    if freq is not None:
      prob *= freq
    else:
      # TODO: Laplace smoothing
      return 0
  return prob

Luego obtenemos **P(A)**, la probabilidad de ocurrencia de un conjunto de palabras:

In [30]:
def p_a(conj_a : list):
  prob = 0
  for category in categories:
    prob += (p_a_cat(conj_a, category) * p_cat[category])
  return prob

Finalmente podemos calcular la probabilidad de que un titular pertenezca a cierta categoría dado su conjunto de palabras:

In [28]:
def p_cat_a(category : str, conj_a : list):
  return p_a_cat(conj_a, category) * p_cat[category] / p_a(conj_a)

Por ejemplo, si tomamos una palabra del vocabulario de Deportes:

In [56]:
sample_word = vocab[categories[dep_idx]][-10]
print(f"P ( cat = 'Deportes' | '{sample_word}' ) = {p_cat_a('Deportes', [sample_word])}")
print(f"P ( cat = 'Noticias destacadas' | '{sample_word}' ) = {p_cat_a('Noticias destacadas', [sample_word])}")
print(f"P ( cat = 'Ciencia y Tecnologia' | '{sample_word}' ) = {p_cat_a('Ciencia y Tecnologia', [sample_word])}")
print(f"P ( cat = 'Entretenimiento' | '{sample_word}' ) = {p_cat_a('Entretenimiento', [sample_word])}")
print(f"P ( cat = 'Destacadas' | '{sample_word}' ) = {p_cat_a('Destacadas', [sample_word])}")

P ( cat = 'Deportes' | 'messi' ) = 0.08695652173913043
P ( cat = 'Noticias destacadas' | 'messi' ) = 0.9130434782608696
P ( cat = 'Ciencia y Tecnologia' | 'messi' ) = 0.0
P ( cat = 'Entretenimiento' | 'messi' ) = 0.0
P ( cat = 'Destacadas' | 'messi' ) = 0.0


## Naive Bayes

Ahora calculamos las probabilidades **P(categoria|A)** para cada categoría y nos quedamos con la de mayor valor:

In [57]:
def naive_bayes(conj_a : list):
  p_cats_a = {}
  for category in categories:
    p_cats_a[category] = p_cat_a(category, conj_a)
  return max(p_cats_a, key=lambda k: p_cats_a[k])

In [58]:
naive_bayes([sample_word])

'Noticias destacadas'

Para la próxima:

*   Añadir suavizado de Laplace
*   Actualizar función de k-fold
*   Cross-validation con test_set